In [16]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder \
     .appName("Test") \
     .getOrCreate()

In [17]:
import urllib.request  # lib that handles URLs
import io
import pandas as pd
pd.options.display.max_rows=250
import numpy as np
from datetime import datetime
from datetime import timedelta

import matplotlib.pyplot as plt
plt.style.use('seaborn')
import matplotlib.ticker as ticker
from matplotlib.ticker import FormatStrFormatter, FuncFormatter

import pyspark.sql.functions as func

In [18]:
df_events = spark.read.parquet("s3://bigdataproject-pr/filtered_events.parquet")
df_events.cache()

DataFrame[GlobalEventID: int, SQLDATE: int, MonthYear: int, Actor1Name: string, Actor2Name: string, EventCode: string, GoldsteinScale: double, NumMentions: int, NumSources: int, NumArticles: int, AvgTone: double, ActionGeo_CountryCode: string, ActionGeo_Fullname: string, ActionGeo_FeatureID: string, SOURCEURL: string]

In [19]:
relevant_events = df_events.where(df_events.SOURCEURL.rlike('trump'))
#relevant_events = relevant_events.orderBy(relevant_events.NumMentions.desc())

In [20]:
import pandas as pd
relevant_events1 = relevant_events.toPandas()
relevant_events1["MonthYear"] = pd.to_datetime(relevant_events1["MonthYear"], format='%Y%m')
relevant_events1["Year"] = relevant_events1.MonthYear.dt.year


/home/hadoop/miniconda/lib/python3.6/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [31]:
#find event with most mentions of Trump
relevant_events1.loc[relevant_events1.NumMentions.max()]

#using the GDELT CAMEO code guide, we see that EventCode 036 means "Express Intent to meet or negotiate"

GlobalEventID                                                    621130008
SQLDATE                                                           20170127
MonthYear                                              2017-01-01 00:00:00
Actor1Name                                                       PRESIDENT
Actor2Name                                                     THERESA MAY
EventCode                                                              036
GoldsteinScale                                                           4
NumMentions                                                              8
NumSources                                                               1
NumArticles                                                              4
AvgTone                                                            1.35849
ActionGeo_CountryCode                                                   UK
ActionGeo_Fullname                                          United Kingdom
ActionGeo_FeatureID      

In [32]:
relevant_events1.loc[relevant_events1.NumArticles.max()]

#the Trump-related event with the most articles, as we can see involves the immigrant crisis
#EventCode 173 means "Arrest, detain, or charge with legal action"

GlobalEventID                                                    621129063
SQLDATE                                                           20170127
MonthYear                                              2017-01-01 00:00:00
Actor1Name                                                         MIGRANT
Actor2Name                                                            None
EventCode                                                              173
GoldsteinScale                                                          -5
NumMentions                                                              5
NumSources                                                               1
NumArticles                                                              5
AvgTone                                                           -3.75114
ActionGeo_CountryCode                                                   MX
ActionGeo_Fullname                           Guadalupe, Nuevo LeóX, Mexico
ActionGeo_FeatureID      

In [21]:
relevant_events1 = relevant_events1[relevant_events1["Year"] >= 2017]

In [22]:
df = relevant_events1[["NumMentions", "ActionGeo_CountryCode", "AvgTone", "Year"]]

In [23]:
df1 = relevant_events1.groupby(["ActionGeo_CountryCode", "Year"]).agg({"NumMentions": "count", "AvgTone": "mean"}).unstack()

In [24]:
df1 = df1.sort_values([("NumMentions", 2017),("NumMentions", 2018)], ascending = False)

In [25]:
top_10 = df1[:10]

In [26]:
top_10 = top_10.drop("NumMentions", axis=1)
top_10 = top_10["AvgTone"]

In [27]:
pip install pandas-bokeh

Note: you may need to restart the kernel to use updated packages.


In [28]:
import pandas as pd
import pandas_bokeh
pandas_bokeh.output_notebook()

Loading BokehJS ...

In [29]:
top_10.plot_bokeh.bar(
    ylabel="AvgTone", 
    title="Sentiment for Top 10 Countries by Trump Mentions", 
    alpha=0.6)

Figure(id='1191', ...)

In [33]:
spark.stop()